In [3]:
# notebooks/sentiment_analysis.ipynb

import pandas as pd
import sys
import os

# Add 'src' folder to path (parent folder is project root)
sys.path.append(os.path.abspath(".."))

# ✅ Now import from src package
from src.sentiment import analyze_sentiments, aggregate_sentiments

# Load the reviews
df = pd.read_csv("../data/all_banks_reviews.csv")

# Run sentiment analysis
df_with_sentiment = analyze_sentiments(df)
df_with_sentiment.to_csv("../data/all_banks_reviews_with_sentiment.csv", index=False)

# Aggregate results
agg_df = aggregate_sentiments(df_with_sentiment)
agg_df.to_csv("../data/aggregated_sentiments_by_bank_and_rating.csv", index=False)

agg_df.head()


c:\Users\soswo\OneDrive\Desktop\projects\Customer-Experience-Analytics-for-Fintech-Apps\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\soswo\OneDrive\Desktop\projects\Customer-Experience-Analytics-for-Fintech-Apps\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\soswo\.cache\huggingface\hub\models--distilbert-base-uncase

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.